### Assignment 1 - K Means Clustering

Using the Python code provided by the Machine Learning in Action book (chapter 10), adapt the code to perform a K-Means Clustering on the Palmer Penguin dataset.

Document each step of the code to demonstrate you understand what each block of code does. 

Display a graph using the Elbow Method to determine an appropriate K.

Document each step of the code to demonstrate you understand what each block of code does.

Feel free to experiment with alternative measurements. Also feel free to use my example for the code to get the Penguin data and explore it.

Document each step of the code to demonstrate you understand what each block of code does. 

In [43]:
# ========= I rewrote most of the code so i can better understand it =========

# Import Libraries
import numpy as np
import pandas as pd
import math

In [44]:
# Loads data from a file into a clean pandas dataframe
def load_file_to_dataframe(fileName, delimiter):
    # Read the CSV file into a DataFrame
    data_df = pd.read_csv(fileName, sep = delimiter)
    
    # Get rid of spaces on string columns
    for column in data_df.columns:
        # Look for columns that are strings
        if data_df[column].dtype == 'object':
            # Get rid of white space
            data_df[column] = data_df[column].str.strip()
    
    # Show data types
    display(data_df.dtypes) 
    
    # Show cleaned dataframe
    return data_df

In [45]:
load_file_to_dataframe('penguins.csv', ',')

studyName            object
SampleNumber          int64
Species              object
Region               object
Island               object
Stage                object
IndividualID         object
ClutchCompletion     object
DateEgg              object
CulmenLength_mm     float64
CulmenDepth_mm      float64
FlipperLength_mm    float64
BodyMass_g          float64
Sex                  object
Delta15N            float64
Delta13C            float64
Comments             object
dtype: object

,studyName,SampleNumber,Species,Region,Island,Stage,IndividualID,ClutchCompletion,DateEgg,CulmenLength_mm,CulmenDepth_mm,FlipperLength_mm,BodyMass_g,Sex,Delta15N,Delta13C,Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,11/11/07,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,11/11/07,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,11/16/07,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,NaN
3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,11/16/07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adult not sampled.
4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,11/16/07,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,PAL0910,120,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N38A2,No,12/1/09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,PAL0910,121,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N39A1,Yes,11/22/09,46.8,14.3,215.0,4850.0,FEMALE,8.41151,-26.13832,NaN
341,PAL0910,122,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N39A2,Yes,11/22/09,50.4,15.7,222.0,5750.0,MALE,8.30166,-26.04117,NaN
342,PAL0910,123,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N43A1,Yes,11/22/09,45.2,14.8,212.0,5200.0,FEMALE,8.24246,-26.11969,NaN


In [50]:
# Calulate the Ecludian Distance between two vectors
def calculate_eclud_dist(vecA, vecB):
    
    # Convert lists to numpy arrays for below code to work
    vecA = np.array(vecA)
    vecB = np.array(vecB)
    
    # Get the difference between two factors
    diff = vecA - vecB
    
    # Square each element of the difference vector
    squared_diff = np.power(diff, 2)
    
    # Sum up all the squared differences to get the squared distance
    squared_dist = sum(squared_diff)
    
    # Take the square root of the squared distance to get the Euclidean distance
    distance = np.sqrt(squared_dist)
    
    # Access the first element of the resulting array to get the distance value
    return distance

In [51]:
vecA = [1, 2]
vecB = [4, 6]

calculate_eclud_dist(vecA,vecB)

5.0

In [67]:
# Get the centroids
def randCent(dataSet, k):
    # Filter to include only numeric columns
    numeric_dataSet = dataSet.select_dtypes(include=[np.number])

        # Get the number of columns out of (rows,columns)
    n = numeric_dataSet.shape[1]
    
        # Initialize centroids with zeros
            # Create an array full of zeroes 
                # k = # of rows and n = # of columns
    centroids = np.zeros((k, n))  
    
    # For each feature (column in the dataset)
    for column in range(n):
        # Find the minimum value in the current column
        min_column_value = numeric_dataSet.iloc[:, column].min()

        # Calculate the range of values in the current column
        range_of_column = numeric_dataSet.iloc[:, column].max() - min_column_value

        # Generate k random values within this range
        random_values = np.random.rand(k)

        # Adjust the random values to fit within the column's actual data range
        adjusted_random_values = random_values * range_of_column

        # Shift the adjusted values to start from the minimum column value
        adjusted_random_values = min_column_value + adjusted_random_values

        # Assign these adjusted random values to the centroids for the current feature
        centroids[:, column] = adjusted_random_values

    return centroids

In [68]:
data_df = pd.read_csv('penguins.csv', sep = ',')

randCent(data_df,3)

array([[  58.40167479,   50.68084118,   14.57757645,  206.52750731,
        4932.47823113,    9.52713885,  -25.2631042 ],
       [  90.0084297 ,   33.65065351,   19.13292188,  177.53433865,
        3149.07806674,    8.27744745,  -23.83948952],
       [ 123.83973588,   35.16785066,   20.10380904,  228.29425154,
        4278.29405109,    9.90583241,  -26.49909713]])

In [ ]:
# This function is provided by Peter Harrington via his GitHub page
# You have to modify it to make it work
# And document what each line is doing with inline comments
def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A==cent)[0]]
            centroids[cent,:] = mean(ptsInClust, axis=0)
    return centroids, clusterAssment

In [ ]:
# I modified the following code from a different source.
# The function showPlt() should take your dataset, compute the K clusters, and plot the results
# It should work, and you can use it as written.
# You do not have to provide inline comments for this function.
import matplotlib
import matplotlib.pyplot as plt
def showPlt(datMat, alg=kMeans, numClust=3):
    myCentroids, clustAssing = alg(datMat, numClust)
    fig = plt.figure()
    rect=[0.1,0.1,0.8,0.8]
    scatterMarkers=['s', 'o', '^', '8', 'p', \
                    'd', 'v', 'h', '>', '<']
    axprops = dict(xticks=[], yticks=[])
    ax0=fig.add_axes(rect, label='ax0', **axprops)
    ax1=fig.add_axes(rect, label='ax1', frameon=False)
    for i in range(numClust):
        ptsInCurrCluster = datMat[np.nonzero(clustAssing[:,0].A==i)[0],:]
        markerStyle = scatterMarkers[i % len(scatterMarkers)]
        ax1.scatter(ptsInCurrCluster[:,0].flatten().A[0], ptsInCurrCluster[:,1].flatten().A[0], marker=markerStyle, s=90)
    ax1.scatter(myCentroids[:,0].flatten().A[0], myCentroids[:,1].flatten().A[0], marker='+', s=300)
    plt.show()